# Smart Urban System - NLP Module Development


## Phase 1: Sources Loading and data extraction and exploration


In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Add src to path
src_path = Path("../src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from preprocessor import SmartUrbanPreprocessor

print("Environment checked. Loading Preprocessor...")

Environment checked. Loading Preprocessor...


## Phase 2: Corpus Acquisition & Preprocessing

This notebook handles the loading, normalization, and cleaning of the heterogeneous corpus.


### 1. Load Raw Data


In [2]:
processor = SmartUrbanPreprocessor()
raw_data = processor.load_data()

for source, df in raw_data.items():
    print(f"{source}: {len(df)} rows")

Initializing NLP models...
Loading Stores Complaints...
Loading Financial Complaints...
Loading University Complaints...
Loading Amazon Reviews...
Loading News Sentiments...
Falling back to ISO-8859-1 encoding for news
Removing non-UTF-8 characters from news text
Removing non-UTF-8 characters from news sentiment
Converted news to UTF-8
stores: 81883 rows
financial: 162421 rows
university: 1005 rows
amazon: 30000 rows
news: 4846 rows


In [3]:
{k: v.shape for k, v in raw_data.items()}

{'stores': (81883, 24),
 'financial': (162421, 3),
 'university': (1005, 8),
 'amazon': (30000, 9),
 'news': (4846, 2)}

In [4]:
stores = raw_data["stores"]
financial = raw_data["financial"]
university = raw_data["university"]
reviews = raw_data["amazon"]
news = raw_data["news"]

dfs = {
    "stores": stores,
    "financial": financial,
    "university": university,
    "reviews": reviews,
    "news": news,
}

#### 1.1 Preliminar EDA


In [5]:
{k: df.head(2) for k, df in dfs.items()}

{'stores':       ID_EXP FECHA_INGRESO            FECHA_FIN      FECHA DE CIERRE  \
 0  2022_3584    2022-01-03  2022-08-15 00:00:00  2022-08-15 00:00:00   
 1  2022_2645    2022-01-03  2022-03-29 00:00:00  2022-03-29 00:00:00   
 
                TIPO_CONCILIACION ESTADO_PROCESAL  \
 0  Turnada a Concil Person p/seg      Conciliada   
 1  Turnada a Concil Person p/seg   Desistimiento   
 
                             PROVEEDOR                    NOMBRE_COMERCIAL  \
 0  DIVERSIDAD PARA EL HOGAR, SA DE CV  DIVERSIDAD PARA EL HOGAR, SA DE CV   
 1                        PROMEDIO 100                        PROMEDIO 100   
 
                                     GIRO  \
 0                              MUEBLERÍA   
 1  ESCUELA DE EDUCACIÓN PRIMARIA PRIVADA   
 
                                   SECTOR  ... COSTO BIEN SERVICIO  \
 0                               MUEBLERO  ...               16878   
 1  ESCUELA DE EDUCACIÓN PRIMARIA PRIVADA  ...             51111.8   
 
   MONTO RECLAMADO MONT

In [6]:
{k: (df.shape, df.dtypes) for k, df in raw_data.items()}

{'stores': ((81883, 24),
  ID_EXP                         object
  FECHA_INGRESO          datetime64[ns]
  FECHA_FIN                      object
  FECHA DE CIERRE                object
  TIPO_CONCILIACION              object
  ESTADO_PROCESAL                object
  PROVEEDOR                      object
  NOMBRE_COMERCIAL               object
  GIRO                           object
  SECTOR                         object
  ODECO                          object
  ESTADO_UA                      object
  TIPO_RECLAMACION               object
  MOTIVO_RECLAMACION             object
  COSTO BIEN SERVICIO            object
  MONTO RECLAMADO               float64
  MONTO RECUPERADO               object
  PROCEDIMIENTO                  object
  BIEN O SERV                    object
  MEDIO INGRESO                  object
  TIPO PROD                      object
  MODALIDAD COMPRA               object
  MODALIDAD PAGO                 object
  PROB ESPECIAL                  object
  dtype: object

In [7]:
# Detailed EDA for each dataset
for name, df in dfs.items():
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Count': missing,
        'Percentage': missing_pct
    })
    display(missing_df[missing_df['Count'] > 0])
    
    # Basic statistics for numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\nNumeric Statistics:")
        display(df[numeric_cols].describe())
    
    # Unique values for categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"\nCategorical Columns - Unique Value Counts:")
        for col in categorical_cols[:5]:  # Show first 5 categorical columns
            unique_count = df[col].nunique()
            print(f"  {col}: {unique_count} unique values")
            if unique_count <= 10:
                print(f"    Values: {df[col].unique()}")
    
    # Sample of data
    print(f"\nSample Data (first 3 rows):")
    display(df.head(3))

# Summary comparison across datasets
print(f"\n\n{'='*60}")
print("DATASET COMPARISON SUMMARY")
print(f"{'='*60}")

comparison_data = []
for name, df in dfs.items():
    comparison_data.append({
        'Dataset': name,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Numeric Columns': len(df.select_dtypes(include=[np.number]).columns),
        'Categorical Columns': len(df.select_dtypes(include=['object']).columns),
        'Missing Values': df.isnull().sum().sum(),
        'Missing %': round(df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100, 2)
    })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)



Missing Values:


,Count,Percentage
TIPO_RECLAMACION,21254,25.96
MOTIVO_RECLAMACION,21254,25.96
COSTO BIEN SERVICIO,212,0.26
MONTO RECLAMADO,50706,61.92
MONTO RECUPERADO,50702,61.92
MODALIDAD PAGO,282,0.34



Numeric Statistics:


,MONTO RECLAMADO
count,3.117700e+04
mean,1.644598e+04
std,1.112201e+05
min,0.000000e+00
25%,0.000000e+00
50%,2.290000e+02
75%,7.976000e+03
max,8.247000e+06



Categorical Columns - Unique Value Counts:
  ID_EXP: 76001 unique values
  FECHA_FIN: 292 unique values
  FECHA DE CIERRE: 293 unique values
  TIPO_CONCILIACION: 7 unique values
    Values: ['Turnada a Concil Person p/seg' 'Turnada a Concil Medios Elec.'
 'Turnada a Concil Resi p/ segui' 'Turnada a Concil Telefónica'
 'Turnada a Concil Domiciliaria' 'En Proceso' 'Cancelada']
  ESTADO_PROCESAL: 6 unique values
    Values: ['Conciliada' 'Desistimiento' 'En Trámite' 'No Conciliada' 'Improcedente'
 'Cancelada']

Sample Data (first 3 rows):


...

,Count,Percentage



Categorical Columns - Unique Value Counts:
  sentiment: 3 unique values
    Values: ['neutral' 'negative' 'positive']
  text: 4838 unique values

Sample Data (first 3 rows):


,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...




DATASET COMPARISON SUMMARY


,Dataset,Rows,Columns,Numeric Columns,Categorical Columns,Missing Values,Missing %
0,stores,81883,24,1,22,144410,7.35
1,financial,162421,3,1,2,10,0.00
2,university,1005,8,4,4,0,0.00
3,reviews,30000,9,2,7,1,0.00
4,news,4846,2,0,2,0,0.00


### 2. Unify Schema


In [8]:
unified_df = processor.normalize(raw_data)
print(unified_df.shape)
display(unified_df.head(5))
print(unified_df.dtypes)

(280155, 5)


,id,text,source,language,sentiment_score
0,de_0784695,"Leider, leider nach einmal waschen ausgebliche...",amazon,de,bad
1,de_0759207,zunächst macht der Anker Halter einen soliden ...,amazon,de,bad
2,de_0711785,Siegel sowie Verpackung war beschädigt und war...,amazon,de,bad
3,de_0964430,Habe dieses Produkt NIE erhalten und das Geld ...,amazon,de,bad
4,de_0474538,Die Träger sind schnell abgerissen Reißverschl...,amazon,de,bad


id                 object
text               object
source             object
language           object
sentiment_score    object
dtype: object


In [9]:
# Missing values
missing = unified_df.isnull().sum()
missing_pct = (missing / len(unified_df) * 100).round(2)
missing_df = pd.DataFrame({
    'Count': missing,
})
display(missing_df)
print(unified_df.shape)
working_df = unified_df[unified_df['text'].notna()]
print(working_df.shape)

print("")

# Unique values
categorical_cols = working_df.select_dtypes(include=['object']).columns
for col in categorical_cols:  
    unique_count = working_df[col].nunique()
    print(f"- Column {col}: {unique_count} unique values")
    if unique_count <= 10:
        print(f"\t- Values: {working_df[col].unique()}")


,Count
id,0
text,1
source,0
language,0
sentiment_score,0


(280155, 5)
(280154, 5)

- Column id: 192420 unique values
- Column text: 273270 unique values
- Column source: 5 unique values
	- Values: ['amazon' 'stores' 'financial' 'university' 'news']
- Column language: 6 unique values
	- Values: ['de' 'en' 'es' 'fr' 'ja' 'zh']
- Column sentiment_score: 3 unique values
	- Values: ['bad' 'neutral' 'good']


### 3. Cleaning & Preprocessing


In [10]:
clean_df = processor.process_corpus(unified_df, apply_spell_correction=False)
# spell_correction_clean_df = processor.process_corpus(unified_df, apply_spell_correction=True)
display(clean_df.head())
# display(spell_correction_clean_df.head())

Processing corpus with 280155 documents...
Filtering by language (en/es)...
Removed 20000 documents with unsupported languages
Removing duplicates...
Removed 6884 duplicate entries
Cleaning text...
Tokenizing and lemmatizing...


Processing: 100%|██████████| 253271/253271 [10:34<00:00, 399.38it/s]  


Processing complete. Final corpus size: 253271 documents


,id,text,source,language,sentiment_score,clean_text,tokens,lemmas,pos_tags,entities,lemmas_text
5000,en_0199937,"These are AWFUL. They are see through, the fab...",amazon,en,bad,these are awful they are see through the fabri...,"[these, are, awful, they, are, see, through, t...","[these, be, awful, they, be, see, through, the...","[DET, AUX, ADJ, PRON, AUX, VERB, ADP, DET, NOU...","[{'text': 'don', 'label': 'PERSON', 'start': 2...",these be awful they be see through the fabric ...
5001,en_0863335,I bought 4 and NONE of them worked. Yes I used...,amazon,en,bad,i bought 4 and none of them worked yes i used ...,"[i, bought, 4, and, none, of, them, worked, ye...","[I, buy, 4, and, none, of, they, work, yes, I,...","[PRON, VERB, NUM, CCONJ, NOUN, ADP, PRON, VERB...","[{'text': '4', 'label': 'CARDINAL', 'start': 9...",I buy 4 and none of they work yes I use new ba...
5002,en_0565010,On first use it didn't heat up and now it does...,amazon,en,bad,on first use it didn't heat up and now it does...,"[on, first, use, it, did, n't, heat, up, and, ...","[on, first, use, it, do, not, heat, up, and, n...","[ADP, ADJ, NOUN, PRON, AUX, PART, VERB, ADP, C...","[{'text': 'first', 'label': 'ORDINAL', 'start'...",on first use it do not heat up and now it do n...
5003,en_0963290,You want an HONEST answer? I just returned fro...,amazon,en,bad,you want an honest answer i just returned from...,"[you, want, an, honest, answer, i, just, retur...","[you, want, an, honest, answer, I, just, retur...","[PRON, VERB, DET, ADJ, NOUN, PRON, ADV, VERB, ...","[{'text': 'first', 'label': 'ORDINAL', 'start'...",you want an honest answer I just return from u...
5004,en_0238156,The glue works fine but the container is impos...,amazon,en,bad,the glue works fine but the container is impos...,"[the, glue, works, fine, but, the, container, ...","[the, glue, work, fine, but, the, container, b...","[DET, NOUN, VERB, ADV, CCONJ, DET, NOUN, AUX, ...",[],the glue work fine but the container be imposs...


### 4. Analysis & Distribution


In [ ]:
# TODO add some vision analysis

In [14]:
# Save processed data
output_path = Path("../data/processed_corpus.csv")
clean_df.to_csv(output_path, index=False)
# spell_correction_clean_df.to_csv(Path("../data/processed_corpus_spell_corrected.csv"), index=False)
print(f"Saved to {output_path}")

Saved to ..\data\processed_corpus.csv
